<a href="https://colab.research.google.com/github/askolesov0608/SJK006-LangModel/blob/main/Copy_of_7th_Deliverable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DELIVERABLE: FINE-TUNING AND ADAPTATION OF LANGUAGE MODELS

In this exercise the aim is to perform transfer learning from English to Spanish in the domain of sentiment analysis in opinions about movies.
It is recommended to use a cross-lingual model in order to facilitate the learning transfer.

- `xlm-roberta-base`
- `xlm-roberta-large`

In [1]:
#pip install --upgrade bigframes
!pip install bigframes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.8 MB/s eta 0:00:00


In [2]:

!pip install ibis-framework

#pip install --upgrade ibis-framework

Choose between transformers or simpletransformers

In [3]:
!pip install transformers[torch]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 2.7 MB/s eta 0:00:00


In [4]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


## Loading, exploring and transforming data for tranformers

Get the data of opinions about movies from IMDB dataset (parquet format)

In [5]:
!wget -O imdb_train.parquet "https://huggingface.co/datasets/imdb/resolve/refs%2Fconvert%2Fparquet/plain_text/train/0000.parquet"
!wget -O imdb_test.parquet  "https://huggingface.co/datasets/imdb/resolve/refs%2Fconvert%2Fparquet/plain_text/test/0000.parquet"

--2024-02-12 14:35:11--  https://huggingface.co/datasets/imdb/resolve/refs%2Fconvert%2Fparquet/plain_text/train/0000.parquet
Resolving huggingface.co (huggingface.co)... 18.172.134.4, 18.172.134.124, 18.172.134.88, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/datasets/imdb/db47d16b5c297cc0dd625e519c81319c24c9149e70e8496de5475f6fa928342c?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%270000.parquet%3B+filename%3D%220000.parquet%22%3B&Expires=1708007711&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwODAwNzcxMX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9kYXRhc2V0cy9pbWRiL2RiNDdkMTZiNWMyOTdjYzBkZDYyNWU1MTljODEzMTljMjRjOTE0OWU3MGU4NDk2ZGU1NDc1ZjZmYTkyODM0MmM%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=fBYJ-2D1rJb5abM5-4BkqTq054SLzYWRd-RdhdRCifQrNvc-sLBYtznwcinFlY%7EdoranQ

In [6]:
!ls

imdb_test.parquet  imdb_train.parquet  sample_data


In [7]:
#import pyarrow.parquet as pq

#train_table = pq.read_table("imdb_train.parquet")
##test_table = pq.read_table("imdb_test.parquet")

#train_df = train_table.to_pandas()
#test_df = test_table.to_pandas()

In [8]:
import pandas as pd
train_df = pd.read_parquet("imdb_train.parquet")
test_df = pd.read_parquet("imdb_test.parquet")

In [9]:
# Просмотр информации о датасетах
print(train_df.info())
print(test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    25000 non-null  object
 1   label   25000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 390.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    25000 non-null  object
 1   label   25000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 390.8+ KB
None


In [10]:
# Просмотр первых записей обучающего датасета
print(train_df.head())

                                                text  label
0  I rented I AM CURIOUS-YELLOW from my video sto...      0
1  "I Am Curious: Yellow" is a risible and preten...      0
2  If only to avoid making this type of film in t...      0
3  This film was probably inspired by Godard's Ma...      0
4  Oh, brother...after hearing about this ridicul...      0


In [11]:
# Распределение классов в обучающем датасете
print(train_df['label'].value_counts(normalize=True))

0    0.5
1    0.5
Name: label, dtype: float64


In [12]:
##Very Very LONG
from transformers import AutoTokenizer

# Загрузка токенизатора
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')

# Функция для токенизации датасета
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [ ]:
##Thiss is Shoter
from transformers import AutoTokenizer

model_name = "microsoft/MiniLM-L12-H384-uncased" #recommended: microsoft/MiniLM-L12-H384-uncased
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Функция для токенизации датасета
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)

In [13]:
# Токенизация данных
train_tokenized = tokenize_function(train_df.to_dict(orient='list'))
test_tokenized = tokenize_function(test_df.to_dict(orient='list'))

In [14]:
import torch

# Преобразование токенизированных данных в тензоры PyTorch
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Создание объектов датасета для обучения и тестирования
train_dataset = IMDbDataset(train_tokenized, train_df['label'].tolist())
test_dataset = IMDbDataset(test_tokenized, test_df['label'].tolist())

## Prepare the Trainer from a cross-lingual pretrained model

In [15]:
#IMDB является бинарной классификацией, количество меток классов (num_labels) будет равно 2.
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=2)


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
#2: Настройка аргументов обучения
training_args = TrainingArguments(
    output_dir='./results',          # Директория для сохранения результатов
    num_train_epochs=5,              # Количество эпох обучения
    per_device_train_batch_size=48,   # Размер батча для обучения
    per_device_eval_batch_size=16,   # Размер батча для валидации
    warmup_steps=500,                # Количество шагов разогрева
    weight_decay=0.01,               # Вес уменьшения
    logging_dir='./logs',            # Директория для логов
    logging_steps=10,                # Частота логирования
    evaluation_strategy="epoch",     # Стратегия оценки
    save_strategy="epoch",           # Стратегия сохранения модели
    load_best_model_at_end=True,     # Загрузка лучшей модели по окончании обучения
    metric_for_best_model="accuracy" # Метрика для выбора лучшей модели
)


In [33]:
# 3: Определение функции для вычисления метрик
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='binary')
    return {'accuracy': acc, 'f1': f1}

In [34]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

In [35]:
# Запуск обучения
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.223000,0.236538,0.916800,0.920586
2,0.163200,0.164512,0.938480,0.939123
3,0.114800,0.198857,0.936160,0.934862
4,0.063600,0.239792,0.939880,0.940950
5,0.026600,0.297615,0.940880,0.941907


TrainOutput(global_step=2605, training_loss=0.15123920946197866, metrics={'train_runtime': 3645.8703, 'train_samples_per_second': 34.285, 'train_steps_per_second': 0.715, 'total_flos': 3.288888192e+16, 'train_loss': 0.15123920946197866, 'epoch': 5.0})

In [36]:
# Оценка модели на тестовом наборе данных
trainer.evaluate(test_dataset)

{'eval_loss': 0.2976151406764984,
 'eval_accuracy': 0.94088,
 'eval_f1': 0.9419070827765113,
 'eval_runtime': 185.0788,
 'eval_samples_per_second': 135.078,
 'eval_steps_per_second': 8.445,
 'epoch': 5.0}

In [37]:
import os

model_path = 'D:/BDModel/model'
tokenizer_path = 'D:/BDModel/tokenizer'

# Создание директорий, если они еще не существуют
if not os.path.exists(model_path):
    os.makedirs(model_path)
if not os.path.exists(tokenizer_path):
    os.makedirs(tokenizer_path)

# Сохранение модели и токенизатора
model.save_pretrained(model_path)
tokenizer.save_pretrained(tokenizer_path)


('D:/BDModel/tokenizer/tokenizer_config.json',
 'D:/BDModel/tokenizer/special_tokens_map.json',
 'D:/BDModel/tokenizer/sentencepiece.bpe.model',
 'D:/BDModel/tokenizer/added_tokens.json',
 'D:/BDModel/tokenizer/tokenizer.json')

In [38]:
model_path = './BDModel/model'
tokenizer_path = './BDModel/tokenizer'

# Создание директорий, если они еще не существуют
if not os.path.exists(model_path):
    os.makedirs(model_path)
if not os.path.exists(tokenizer_path):
    os.makedirs(tokenizer_path)

# Сохранение модели и токенизатора
model.save_pretrained(model_path)
tokenizer.save_pretrained(tokenizer_path)

('./BDModel/tokenizer/tokenizer_config.json',
 './BDModel/tokenizer/special_tokens_map.json',
 './BDModel/tokenizer/sentencepiece.bpe.model',
 './BDModel/tokenizer/added_tokens.json',
 './BDModel/tokenizer/tokenizer.json')

In [39]:
import zipfile

# Путь к папке BDModel
folder_path = './BDModel'

# Путь к архивному файлу, который вы хотите создать
zip_file_path = './BDModel_archive.zip'

# Создание архива
with zipfile.ZipFile(zip_file_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Рекурсивное добавление всех файлов и подпапок из папки BDModel в архив
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), os.path.join(folder_path, '..')))


In [40]:
#from transformers import AutoTokenizer
#Путь к сохраненному токенизатору
tokenizer_path = 'D:/BDModel/tokenizer'

# Загрузка токенизатора из локальной директории
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)


In [41]:
#from transformers import AutoModelForSequenceClassification

# Путь к сохраненной модели
model_path = 'D:/BDModel/model'

# Загрузка модели из локальной директории
model = AutoModelForSequenceClassification.from_pretrained(model_path)


In [42]:
# Пример текста для классификации
text = "This movie was fantastic! I loved it."

# Токенизация текста
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

# Получение предсказаний от модели
outputs = model(**inputs)

# Преобразование выходных данных модели в вероятности
import torch.nn.functional as F
probs = F.softmax(outputs.logits, dim=-1)

# Вывод вероятностей для каждого класса
print(probs)


tensor([[0.0011, 0.9989]], grad_fn=<SoftmaxBackward0>)


In [46]:
# Пример текста для классификации
text = "This movie bad."

# Токенизация текста
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

# Получение предсказаний от модели
outputs = model(**inputs)

# Преобразование выходных данных модели в вероятности
import torch.nn.functional as F
probs = F.softmax(outputs.logits, dim=-1)

# Вывод вероятностей для каждого класса
print(probs)

tensor([[9.9937e-01, 6.2613e-04]], grad_fn=<SoftmaxBackward0>)


In [47]:
!wget https://krono.act.uji.es/IDIA/criticas_pelis.csv.gz
!gunzip criticas_pelis.csv.gz

--2024-02-12 15:54:25--  https://krono.act.uji.es/IDIA/criticas_pelis.csv.gz
Resolving krono.act.uji.es (krono.act.uji.es)... 150.128.97.37
Connecting to krono.act.uji.es (krono.act.uji.es)|150.128.97.37|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4447654 (4.2M) [application/x-gzip]
Saving to: ‘criticas_pelis.csv.gz’

criticas_pelis.csv. 100%[===================>]   4.24M  4.04MB/s    in 1.1s    

2024-02-12 15:54:27 (4.04 MB/s) - ‘criticas_pelis.csv.gz’ saved [4447654/4447654]



In [49]:
df = pd.read_csv("criticas_pelis.csv", names=["Row", "File", "Text", "Rating"])
df.head()

,Row,File,Text,Rating
0,Row0,File-0,"May, ¿quieres ser mi amigo? May, ¿Quieres ser...",4
1,Row1,File-1,Cómo ponerse en la piel de un kamikaze Es tod...,4
2,Row2,File-10,"Deliciosa comedieta dramática, con tintes rev...",4
3,Row3,File-100,La ironía es el arma de los perdedores y este...,3
4,Row4,File-1000,"Al final, y teniendo en cuenta que esto es el...",3


In [52]:
# Загрузка датасета с указанием названий колонок
df = pd.read_csv("criticas_pelis.csv", names=["Row", "File", "Text", "Rating"])

# Удаление колонок "Row" и "File"
df = df.drop(['Row', 'File'], axis=1)

# Предварительный анализ данных
print(df.head())
print(df.describe())

# Фильтрация нейтральных отзывов и преобразование оценок в бинарные метки
df_filtered = df[df['Rating'] != 3].copy()  # Использование .copy() для избежания предупреждения
df_filtered['Label'] = df_filtered['Rating'].apply(lambda x: 1 if x > 3 else 0)

# Проверка распределения меток после преобразования
print(df_filtered['Label'].value_counts())
print(df_filtered.head())

                                                Text  Rating
0   May, ¿quieres ser mi amigo? May, ¿Quieres ser...       4
1   Cómo ponerse en la piel de un kamikaze Es tod...       4
2   Deliciosa comedieta dramática, con tintes rev...       4
3   La ironía es el arma de los perdedores y este...       3
4   Al final, y teniendo en cuenta que esto es el...       3
            Rating
count  3878.000000
mean      3.048221
std       1.141519
min       1.000000
25%       2.000000
50%       3.000000
75%       4.000000
max       5.000000
1    1351
0    1274
Name: Label, dtype: int64
                                                Text  Rating  Label
0   May, ¿quieres ser mi amigo? May, ¿Quieres ser...       4      1
1   Cómo ponerse en la piel de un kamikaze Es tod...       4      1
2   Deliciosa comedieta dramática, con tintes rev...       4      1
5   Durante buena parte del metraje y solo pude r...       1      0
8   Sus defectos quedan olvidados gracias a las i...       4      1


In [68]:
from sklearn.model_selection import train_test_split

# Разделение данных на тренировочный и валидационный наборы
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df_filtered['Text'], df_filtered['Label'],
    test_size=0.1,  # 10% данных для валидации
    random_state=42  # Задаем seed для воспроизводимости
)

In [69]:
# Токенизация тренировочного набора данных
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=512)

# Токенизация валидационного набора данных
val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True, max_length=512)

In [70]:
# Создание датасетов для тренировки и валидации
class CriticasPelisDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [71]:
train_dataset = CriticasPelisDataset(train_encodings, train_labels.tolist())
val_dataset = CriticasPelisDataset(val_encodings, val_labels.tolist())

In [76]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [83]:
training_args = TrainingArguments(
    output_dir='./results',          # папка для сохранения результатов
    num_train_epochs=10,              # количество эпох обучения
    per_device_train_batch_size=48,  # размер батча для обучения
    per_device_eval_batch_size=48,   # размер батча для валидации
    warmup_steps=500,                # количество шагов разогрева
    weight_decay=0.01,               # коэффициент уменьшения веса
    evaluation_strategy="epoch",     # стратегия оценки
    save_strategy="epoch",           # стратегия сохранения модели
    logging_dir='./logs',            # папка для логов
    logging_steps=10,
    load_best_model_at_end=True,     # загрузка лучшей модели в конце обучения
    metric_for_best_model="f1",      # метрика для определения лучшей модели
    greater_is_better=True           # указывает, что большие значения метрики лучше
)
#metric_for_best_model="accuracy" # Метрика для выбора лучшей модели


In [84]:
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset  # Используем валидационный набор данных
)

In [85]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.001400,0.380107,0.923954,0.925926,0.892857,0.961538
2,0.004800,0.447976,0.923954,0.925373,0.898551,0.953846
3,0.012300,0.559094,0.916350,0.917910,0.891304,0.946154
4,0.000200,0.564360,0.916350,0.918519,0.885714,0.953846
5,0.000300,0.566810,0.920152,0.923077,0.881119,0.969231
6,0.135400,0.284045,0.923954,0.921875,0.936508,0.907692
7,0.141600,0.236732,0.931559,0.929687,0.944444,0.915385
8,0.030600,0.449992,0.923954,0.927536,0.876712,0.984615
9,0.056700,0.451270,0.923954,0.921875,0.936508,0.907692
10,0.089800,0.390966,0.939163,0.939850,0.919118,0.961538


TrainOutput(global_step=500, training_loss=0.04847958060866222, metrics={'train_runtime': 628.7316, 'train_samples_per_second': 37.568, 'train_steps_per_second': 0.795, 'total_flos': 6214683127603200.0, 'train_loss': 0.04847958060866222, 'epoch': 10.0})

In [86]:
# Оценка модели на валидационном наборе данных
eval_results = trainer.evaluate(val_dataset)

In [87]:
# Вывод результатов оценки
print(eval_results)

{'eval_loss': 0.39096590876579285, 'eval_accuracy': 0.9391634980988594, 'eval_f1': 0.9398496240601504, 'eval_precision': 0.9191176470588235, 'eval_recall': 0.9615384615384616, 'eval_runtime': 1.8751, 'eval_samples_per_second': 140.262, 'eval_steps_per_second': 3.2, 'epoch': 10.0}


In [88]:
model.save_pretrained('./fine_tuned_model')
tokenizer.save_pretrained('./fine_tuned_model')


('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/sentencepiece.bpe.model',
 './fine_tuned_model/added_tokens.json',
 './fine_tuned_model/tokenizer.json')

In [89]:
# Загрузка тестового набора данных
test_df = pd.read_parquet("imdb_test.parquet")

# Вывод первых нескольких строк для проверки
print(test_df.head())

# Вывод распределения меток в тестовом наборе данных
print(test_df['label'].value_counts())

# Токенизация тестового набора данных
test_encodings = tokenizer(test_df['text'].tolist(), truncation=True, padding=True, max_length=512, return_tensors="pt")

                                                text  label
0  I love sci-fi and am willing to put up with a ...      0
1  Worth the entertainment value of a rental, esp...      0
2  its a totally average film with a few semi-alr...      0
3  STAR RATING: ***** Saturday Night **** Friday ...      0
4  First off let me say, If you haven't enjoyed a...      0
0    12500
1    12500
Name: label, dtype: int64


In [90]:
class IMDbDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: self.encodings[key][idx] for key in self.encodings}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

test_dataset = IMDbDataset(test_encodings, test_df['label'].tolist())

# Вывод размера тестового набора данных
print("Размер тестового набора данных:", len(test_dataset))

Размер тестового набора данных: 25000


In [ ]:
# Оценка модели на тестовом наборе данных
eval_results = trainer.evaluate(test_dataset)

# Вывод результатов оценки
print("Результаты оценки на тестовом наборе данных:")
print(eval_results)


# PART I: Training with the English dataset

## +Loading, exploring and transforming data for tranformers

## +Prepare the Trainer from a cross-lingual pretrained model

## Split train-validation-test

### Get the tokenizer and the classification model (from_pretrained)

### Prepare the arguments of the trainer and set the trained object

## Fine-tune the model

## Evaluate the model with the test dataset (English)

# Part II: Adaptation to Spanish dataset (criticas_pelis)

Using the same model we have evaluated in the previous part, we are going to re-train it with the Spanish examples.

Take care of using the same labels in the Spanish dataset (0=negative, 1=positive). Neutral opinions (rate=3) will be discarded from the training dataset, but we can use them afterwards to predict their polarity with the resulting model.

## Load, explore and transform the Spanish dataset of Critics

## Prepare the Trainer from the previous trained model

## Fine-tune the model

## Evaluate the model with the test dataset (Spanish)

## Evaluate again the model on the English test dataset